In [ ]:
import boto3

elb = boto3.client('elb')
ec2 = boto3.client('ec2')
autoscaling = boto3.client('autoscaling')

In [ ]:
my_session = boto3.session.Session()
my_region = my_session.region_name
print my_region

In [ ]:
# AMIs are unique per region. Here we will look up our AMI based on our region
amis = {}
amis['us-east-1'] = 'ami-0b33d91d'
amis['us-east-2'] = 'ami-446f3521'
amis['us-west-1'] = 'ami-9fadf8ff'
amis['us-west-2'] = 'ami-7abc111a'
amis['eu-west-1'] = 'ami-a1491ad2'
amis['ca-central-1'] = 'ami-ebed508f'

ami_id = amis[my_region]
print ami_id

In [ ]:
user_data = \
"""#!/bin/bash
yum update -y
yum install httpd -y
cd /var/www/html
echo "<html><h1>Hello cloud gurus</h1></html>" > /var/www/html/index.html
service httpd start"""

In [ ]:
# Make a web security group for the load balancer
response = ec2.create_security_group(
    GroupName='web_sg',
    Description='SG for web ingres'
)

webSGID = response['GroupId']
print 'created security group ', webSGID

# Allow ingress for ssh
response = ec2.authorize_security_group_ingress(
    GroupId=webSGID,
    IpProtocol='tcp',
    FromPort=80,
    ToPort=80,
    CidrIp='0.0.0.0/0'
)

print response

In [ ]:
# Create a launch configuration
response = autoscaling.create_launch_configuration(
    LaunchConfigurationName='sample',
    ImageId=ami_id,
    SecurityGroups=[
        'web_sg',
    ],
    UserData=user_data,
    InstanceType='t2.micro',
)

print response

In [ ]:
# What are the availability zones in the region?
response = ec2.describe_availability_zones()

zones = []
for z in response['AvailabilityZones']:
    zones.append(z['ZoneName'])
    
print zones

In [ ]:
# Create the load balancer
create_elb_response = elb.create_load_balancer(
    LoadBalancerName='sample-lb',
    Listeners=[
        {
            'Protocol': 'HTTP',
            'LoadBalancerPort': 80,
            'InstanceProtocol': 'HTTP',
            'InstancePort': 80
        },
    ],
    AvailabilityZones=[
        zones[0], zones[1]
    ],
    SecurityGroups=[
        webSGID,
    ]
)

print create_elb_response

In [ ]:
# Configure health check for the load balancer
hc_response = elb.configure_health_check(
    LoadBalancerName='sample-lb',
    HealthCheck={
        'Target': 'HTTP:80/index.html',
        'Interval': 20,
        'Timeout': 5,
        'UnhealthyThreshold': 3,
        'HealthyThreshold': 2
    }
)

print hc_response

In [ ]:
# Create autoscaling group 
response = autoscaling.create_auto_scaling_group(
    AutoScalingGroupName='sample',
    LaunchConfigurationName='sample',
    MinSize=1,
    MaxSize=10,
    DesiredCapacity=1,
    DefaultCooldown=120,
    AvailabilityZones=[
        zones[0], zones[1]
    ],
    LoadBalancerNames=[
        'sample-lb',
    ],
    HealthCheckType='ELB',
    HealthCheckGracePeriod=180
)

print response

In [ ]:
response = elb.describe_load_balancers(
    LoadBalancerNames=[
        'sample-lb',
    ]
)

print response

In [ ]:
# Drain it first
response = autoscaling.update_auto_scaling_group(
    AutoScalingGroupName='sample',
    LaunchConfigurationName='sample',
    MinSize=0,
    MaxSize=10,
    DesiredCapacity=0,
    DefaultCooldown=120
)

print response

In [ ]:
# Destroy autoscaling group
response = autoscaling.delete_auto_scaling_group(
    AutoScalingGroupName='sample'
)

print response

In [ ]:
# Destroy the launch config
response = autoscaling.delete_launch_configuration(
    LaunchConfigurationName='sample'
)

print response

In [ ]:
# Destroy the load balancer
response = elb.delete_load_balancer(
    LoadBalancerName='sample-lb'
)

print response

In [ ]:
# Delete the security group
response = ec2.delete_security_group(
    GroupId=webSGID
)

print response